In [1]:
import numpy as np
import os
import pandas as pd
import copy 
import datetime 
data_path = '../../../code_data/shanghai_t2dm/'
process_file = 'nips_23'

# pid_list = [1000 + d for d in range(1, 13)]


In [2]:
names = ['ts', 'glucose_level', 'finger_stick', 'blood_ketone', 'meal', 'insulin_dose_sc', 'hypo_agents', 'insulin_bolus', 'basal',	'insulin_iv',] 
usecols = [0, 1, 2, 3, 4, 6, 7, 8, 9, 10]
pid2df = {}
for x in os.listdir(os.path.join(data_path,
                                  'processing/temp1')):
    if x.endswith('.xlsx'):
        # Prints only text file present in My Folder
        print(x)
        pid = int(x[:4])
        df = pd.read_excel(os.path.join(data_path, 'processing/temp1',x), names=names, usecols=usecols)
        if pid not in pid2df:
            pid2df[pid] = df
        else:
            pid2df[pid] = pd.concat([pid2df[pid], df])

2000_0_20201230.xlsx
2001_0_20201102.xlsx
2001_1_20201117.xlsx
2002_0_20210513.xlsx
2003_0_20210615.xlsx
2004_0_20211028.xlsx
2005_0_20211201.xlsx
2006_0_20211112.xlsx
2007_0_20220108.xlsx
2008_0_20220118.xlsx
2009_0_20211103.xlsx
2010_0_20220111.xlsx
2011_0_20220123.xlsx
2012_0_20220126.xlsx
2013_0_20220123.xlsx
2014_0_20201224.xlsx
2014_1_20210317.xlsx
2015_0_20210203.xlsx
2015_1_20210219.xlsx
2016_0_20201224.xlsx
2017_0_20210102.xlsx
2017_1_20201118.xlsx
2018_0_20210420.xlsx
2019_0_20210513.xlsx
2020_0_20210423.xlsx
2021_0_20211013.xlsx
2022_0_20210419.xlsx
2023_0_20210812.xlsx
2024_0_20210429.xlsx
2025_0_20210506.xlsx
2026_0_20210916.xlsx
2027_0_20210521.xlsx
2028_0_20210426.xlsx
2029_0_20210526.xlsx
2030_0_20210531.xlsx
2031_0_20210601.xlsx
2032_0_20210727.xlsx
2033_0_20210615.xlsx
2034_0_20210624.xlsx
2035_0_20210629.xlsx
2036_0_20210803.xlsx
2037_0_20201221.xlsx
2038_0_20210608.xlsx
2039_0_20210615.xlsx
2040_0_20210729.xlsx
2041_0_20210813.xlsx
2042_0_20211019.xlsx
2043_0_202105

In [3]:
import re
def change_meal(x):
    if pd.notna(x):
        pattern = re.compile(r'[-+]?[0-9]*\.?[0-9]+')   # find_number
        result = pattern.findall(x)
        if len(result) == 0:
            return np.nan
        return np.array(result, dtype=float).sum()
for pid in pid2df:
    pid2df[pid]['meal'] = pid2df[pid]['meal'].apply(change_meal)

In [4]:
for pid in pid2df:
    pid2df[pid]['meal'] = pid2df[pid]['meal'].astype(np.float64)

In [5]:
def change_insulin(x):
    if pd.notna(x):
        if type(x) != str:
            return x
        pattern = re.compile(r'[-+]?[0-9]*\.?[0-9]+\s*IU')   # find_number
        result = pattern.findall(x)
        if len(result) == 0:
            return np.nan

        result = [res[:-3] for res in result]
        final = np.array(result, dtype=float).sum()
        return final
for pid in pid2df:
    pid2df[pid]['insulin_dose_sc'] = pid2df[pid]['insulin_dose_sc'].apply(change_insulin)
    # pid2df[pid]['hypo_agents'] = pid2df[pid]['hypo_agents'].apply(change_insulin)
    pid2df[pid]['insulin_bolus'] = pid2df[pid]['insulin_bolus'].apply(change_insulin)
    pid2df[pid]['basal'] = pid2df[pid]['basal'].apply(change_insulin)
    pid2df[pid]['insulin_iv'] = pid2df[pid]['insulin_iv'].apply(change_insulin)

In [6]:
def nan_sum(x):
        if pd.notna(x[0]) and pd.notna(x[1]):
            return x[0] + x[1]
        elif pd.notna(x[0]):
            return x[0]
        elif pd.notna(x[1]):
            return x[1]
        else:
            return np.NaN
for pid in pid2df:
    pid2df[pid]['insulin_dose_sc'] = pid2df[pid]['insulin_dose_sc'].astype(np.float64)
    pid2df[pid]['insulin_bolus'] = pid2df[pid]['insulin_bolus'].astype(np.float64)
    pid2df[pid]['basal'] = pid2df[pid]['basal'].astype(np.float64)
    pid2df[pid]['insulin_iv'] = pid2df[pid]['insulin_iv'].astype(np.float64)
    pid2df[pid]['bolus'] = pid2df[pid][['insulin_dose_sc', 'insulin_bolus']].apply(nan_sum, axis=1)
    

In [7]:
def change_non_insulin(x):
    if pd.notna(x):
        if type(x) != str:
            return x
        pattern = re.compile(r'[-+]?[0-9]*\.?[0-9]+')   # find_number
        result = pattern.findall(x)
        if len(result) == 0:
            return np.nan
        final = np.array(result, dtype=float).sum()
        return final
for pid in pid2df:
    pid2df[pid]['hypo_agents'] = pid2df[pid]['hypo_agents'].apply(change_non_insulin)


In [8]:
for pid in pid2df:
    pid2df[pid]['hypo_agents'] = pid2df[pid]['hypo_agents'].astype(np.float64)

In [9]:
pid2regular_df = {}
pid_list = list(pid2df.keys())
for pid in pid_list:
    df = pid2df[pid]
    df = df.set_index('ts')
    df = df.sort_index()
    pid2regular_df[pid] = df
    print(f'{pid}, total_points:{len(df)}, start:{df.index[0]}, end:{df.index[-1]}')


2000, total_points:1339, start:2020-12-30 13:54:00, end:2021-01-13 12:24:00
2001, total_points:2534, start:2020-11-02 09:40:00, end:2020-12-01 07:49:00
2002, total_points:571, start:2021-05-13 16:22:00, end:2021-05-19 14:52:00
2003, total_points:585, start:2021-06-15 17:20:00, end:2021-06-21 19:20:00
2004, total_points:484, start:2021-10-28 14:28:00, end:2021-11-02 15:13:00
2005, total_points:691, start:2021-12-01 15:55:00, end:2021-12-08 20:25:00
2006, total_points:480, start:2021-11-12 15:20:00, end:2021-11-17 15:05:00
2007, total_points:831, start:2022-01-08 16:46:00, end:2022-01-17 08:16:00
2008, total_points:533, start:2022-01-18 16:29:00, end:2022-01-24 05:29:00
2009, total_points:494, start:2021-11-03 09:43:00, end:2021-11-08 12:58:00
2010, total_points:539, start:2022-01-11 17:44:00, end:2022-01-17 08:14:00
2011, total_points:462, start:2022-01-23 16:58:00, end:2022-01-28 12:13:00
2012, total_points:288, start:2022-01-26 12:33:00, end:2022-01-29 12:18:00
2013, total_points:350,

In [10]:
# add time features

def add_time_attributes(pid2data):
    for pid in pid2data:
        data = pid2data[pid]

        temp = data.index.to_frame().iloc[:, 0].dt
        
        data['day_of_week'] = temp.dayofweek
        data['day_of_week'] = data['day_of_week'].astype(np.float64)

        data['hour'] = temp.hour
        data['hour'] = data['hour'].astype(np.float64)

        data['minute'] = temp.minute
        data['minute'] = data['minute'].astype(np.float64)

        
        data['timestamp'] = temp.hour * 3600 +\
                                temp.minute * 60 +\
                                temp.second
        data['timestamp'] = data['timestamp'].astype(np.float64)
        
        # new ————————————————————————
        seconds_in_day = 24*60*60

        data['sin_time'] = np.sin(2 * np.pi * data.timestamp / seconds_in_day)
        data['cos_time'] = np.cos(2 * np.pi * data.timestamp / seconds_in_day)
        data['sin_time'].astype(np.float64)
        data['cos_time'].astype(np.float64)
        # end ______________________
        
        data['datastamp'] = temp.year * 10000 + temp.month * 100 + temp.day
        
        
    return pid2data
pid2regular_df = add_time_attributes(pid2regular_df)
for pid in pid2regular_df:
    data = pid2regular_df[pid]
    data = data.loc[pd.notna(data['glucose_level'])]
    pid2regular_df[pid] = data

In [11]:
def f(x):
    return 1.509 * ((np.log(x))**1.084 - 5.381)

def cal_LBGI(x):
    x = f(x) if f(x) <= 0 else 0
    return 10 * x ** 2

def cal_HBGI(x):
    x = f(x) if f(x) > 0 else 0
    return 10 * x ** 2
count2count_list = {}
count2count_list['No. of Days'] = []
count2count_list['CV'] = []
count2count_list['TIR'] = []
count2count_list['TBR'] = []
count2count_list['TAR'] = []

count2count_list['LBGI'] = []
count2count_list['HBGI'] = []

count2count_list['Mean of CGM data'] = []
count2count_list['SD of CGM data'] = []
count2count_list['No. of CGM records'] = []

for pid in pid2regular_df:
    
    glucose_avg_by_day = pid2regular_df[pid][['glucose_level', 'datastamp']].groupby('datastamp').mean()
    
    count2count_list['No. of Days'].append(len(glucose_avg_by_day))
    
    glucose_mean = pid2regular_df[pid]['glucose_level'].mean()
    glucose_std = pid2regular_df[pid]['glucose_level'].std()
    count2count_list['Mean of CGM data'].append(glucose_mean)
    count2count_list['SD of CGM data'].append(glucose_std)
    
    count2count_list['No. of CGM records'].append(len(pid2regular_df[pid]['glucose_level']))
 
    count2count_list['CV'].append((glucose_std / glucose_mean) * 100)

    target_range_min = 70
    target_range_max = 180
    
    glucose_values = pid2regular_df[pid]['glucose_level']
    time_in_range = ((glucose_values >= target_range_min) & (glucose_values <= target_range_max)).sum()
    total_time_points = len(glucose_values)
    tir_percentage = (time_in_range / total_time_points) * 100
    count2count_list['TIR'].append(tir_percentage)
    
    
    time_below_range = ((glucose_values < target_range_min)).sum()
    tbr_percentage = (time_below_range / total_time_points) * 100
    count2count_list['TBR'].append(tbr_percentage)
    
    time_above_range = ((glucose_values > target_range_max)).sum()
    tar_percentage = (time_above_range / total_time_points) * 100
    count2count_list['TAR'].append(tar_percentage)
    
    LBGI = pid2regular_df[pid]['glucose_level'].apply(cal_LBGI)
    HBGI = pid2regular_df[pid]['glucose_level'].apply(cal_HBGI)

    count2count_list['LBGI'].append(LBGI.mean())
    count2count_list['HBGI'].append(HBGI.mean())
    


In [12]:
import xlwt

book = xlwt.Workbook(encoding='utf-8', style_compression=0)
sheet = book.add_sheet('Main', cell_overwrite_ok=True)

row_names = ['No. of Days', 'No. of CGM records', 'Mean of CGM data', 'SD of CGM data', 'TIR', 'TBR','TAR', 'CV', 'LBGI', 'HBGI'] 


for r_idx, r_name in enumerate(row_names):
    sheet.write(r_idx + 3, 0, r_name)
    
    if 'No' not in r_name:
        print_str = f'{np.mean(count2count_list[r_name]):.2f}({np.std(count2count_list[r_name]):.2f})'
    else:
        print_str = f'{int(np.mean(count2count_list[r_name]))}({int(np.std(count2count_list[r_name]))})'
    sheet.write(r_idx + 3, 1, print_str)
book.save(f'shanghai_t2dm.xls')
for k in count2count_list:

   print(k, np.mean(count2count_list[k]), np.std(count2count_list[k]))

No. of Days 12.9 6.221736092120912
CV 28.886470796547684 6.222812125350363
TIR 77.73972492777281 17.413514071624242
TBR 2.5387289845388126 7.440432718664699
TAR 19.721546087688388 17.78153511286719
LBGI 0.9603732754014794 1.9667491443608685
HBGI 4.424469197743713 3.716357742170912
Mean of CGM data 141.0548347901342 29.698714142648075
SD of CGM data 40.894135250560836 12.859746717260816
No. of CGM records 1122.87 580.8089127243142
